# 張量運算

## 載入套件, 顯示版本

In [1]:
import tensorflow as tf

print(tf.__version__)

2.9.1


In [2]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## GPU 記憶體管理 (需要在一開始就設定, 這是跟硬體有關, 所以不能隨便設定)

In [4]:
# 限制 Tensorflow 只能使用 GPU 4GB 記憶體
gpus = tf.config.experimental.list_physical_devices("GPU")

if gpus:
    try:
        # 限制第一顆 GPU 只能使用 4GB 記憶體
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit = 1024*4)]
        )

        # 顯示 GPU 個數
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # 顯示錯誤訊息
        print(e)

1 Physical GPUs, 1 Logical GPUs


## Disable GPU
- 這是 disable 哪邊的 GPU?? 好像沒變

In [5]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [6]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

1 Physical GPUs, 1 Logical GPUs


# 張量運算

## 宣告 tensor 常數 (constant)
- 參數可以是常數, list, numpy array

In [8]:
x = tf.constant([[1, 2]])
print(x)

tf.Tensor([[1 2]], shape=(1, 2), dtype=int32)


## 四則運算

In [9]:
print(x+10)
print(x-10)
print(x*2)
print(x/2)

tf.Tensor([[11 12]], shape=(1, 2), dtype=int32)
tf.Tensor([[-9 -8]], shape=(1, 2), dtype=int32)
tf.Tensor([[2 4]], shape=(1, 2), dtype=int32)
tf.Tensor([[0.5 1. ]], shape=(1, 2), dtype=float64)


## tensor 轉換成 Numpy Array

In [10]:
(x+10).numpy()

array([[11, 12]])

In [11]:
type(x.numpy())

numpy.ndarray

## 使用 tensorflow 函數來做運算

In [12]:
# 轉為負數
print(tf.negative(x))

# 常數, list, Numpy array 均可運算
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))

print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# 混用四則運算符號及 Tensorflow 函數
print(tf.square(2) + tf.square(3))


tf.Tensor([[-1 -2]], shape=(1, 2), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


In [13]:
import numpy as np

y = np.array([[1, 2], [3, 4]])
y

array([[1, 2],
       [3, 4]])

In [14]:
tf.reduce_sum(y, axis = 0)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([4, 6])>

In [15]:
tf.reduce_sum([[1, 2], [3, 4]], axis = 1)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 7])>

## numpy和張量之間是相容的
- tf 的運算可以自動轉換為 numpy，反之亦然。
- 張量使用. NumPy()方法顯式地轉換為NumPy ndarrays。
- 這些轉換通常很便宜，因為陣列和tf張量共享底層的記憶體表示，如果可能的話。
- 然而，並不總是共享底層的，因為tf張量可能駐留在GPU記憶體中，而NumPy陣列總是由主機記憶體支援，並且轉換涉及從GPU到主機記憶體的複製。

In [16]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


## TensorFlow 會自動決定在 cpu or gpu 運算

### gpu 加速

In [17]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

Is there a GPU available: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is the Tensor on GPU #0:  
True


In [18]:
x1 = tf.constant([[1, 2, 3]], dtype = float)
print("x1 是否在 GPU #0 上: ", x1.device.endswith("GPU:0"))     # True
print("x1 是否在 GPU #0 上 (空一格): ", x1.device.endswith("GPU: 0"))     # False

# 設定 x 為均勻分配亂數 3x3
x2 = tf.random.uniform([3, 3])
print("x2 是否在 GPU #0 上: ", x2.device.endswith("GPU:0"))

x3 = x1 + x2
print("x3 是否在 GPU #0 上: ", x3.device.endswith("GPU:0") )

x1 是否在 GPU #0 上:  True
x1 是否在 GPU #0 上 (空一格):  False
x2 是否在 GPU #0 上:  True
x3 是否在 GPU #0 上:  True


## 強制指定在 CPU 或 GPU 上運算

In [20]:
import time

# 計算 10 次時間
def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)

    result = time.time() - start
    print("10 loops: {:0.2f}ms".format(1000*result))

# 強制指定在 cpu 運算
print("On CPU:")
with tf.device("CPU:0"):
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("CPU:0")       # 不能空格 ("CPU: 0") 不對
    time_matmul(x)

# 強制指定在 gpu 運算
if tf.config.list_physical_devices("GPU"):
    print("On GPU: ")
    with tf.device("GPU:0"):
        x = tf.random.uniform([1000, 1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)

On CPU:
10 loops: 37.01ms
On GPU: 
10 loops: 0.00ms


## 資料集
- 這一部分介紹 tf.data.Dataset API

### 建立dataset

In [21]:
# 根據slice建立dataset
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

import os

path = "./text.txt"
# 生成儲存一個txt文字
with open(path, 'w') as f:
  f.write("""Line 1
     Line 2
Line 3
  """)
# 根據textline建立dataset
ds_file = tf.data.TextLineDataset(path)
print(ds_tensors)
print(ds_file)

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
<TextLineDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


## 使用map、batch、shuffle等函式對dataset進行轉換
- map:將一種運算對映到資料庫上的元素，運算可以自行定義
- batch:對資料進行打包，如下例中的2指打包後每份的元素個數為2
- shuffle：隨機打亂資料庫中的元素，其中的buffer_size不是很好解釋，
- 可參見https://juejin.im/post/5b855d016fb9a01a1a27d035

In [22]:
# 對ds_tensors 中的資料平方後，進行打亂（buffersize為2），然後進行分包
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

# 對ds_file中的元素進行分包
ds_file = ds_file.batch(2)
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x.numpy())
print('\nElements in ds_file:')
for x in ds_file:
  print(x)

Elements of ds_tensors:
[4 9]
[16  1]
[36 25]

Elements in ds_file:
tf.Tensor([b'Line 1' b'     Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)
